In [45]:
import pickle
import numpy as np

from tqdm import tqdm
from dataclasses import dataclass
from scipy.linalg import solve, block_diag

from dynamics import Dynamics, coe2rv

In [46]:
@dataclass
class SimulationConfig:
    # Simulation parameters
    dt: float = 10.0  # Time step [s] (1, 5, 10, 20, 30, 60)
    K: int = (
        6
        * 395  # Simulation duration in timesteps (57000, 11400, 5700, 2850, 1900, 950)
    )
    H: int = 2  # Window size [timesteps]
    seed: int = 42  # Random seed for reproducibility

    # Network parameters
    N: int = 10  # Number of systems
    number_of_chiefs: int = 1  # Number of chiefs
    number_of_deputies: int = N - number_of_chiefs  # Number of deputies
    n_p: int = 3  # Position vector dimension
    n_v: int = 3  # Velocity vector dimension
    n_x: int = n_p + n_v  # State vector dimension
    n: int = N * n_x  # Global state vector dimension
    o_chief: int = 3  # Chief observation vector dimension
    o_deputy: int = 3  # Deputy observation vector dimension
    o: int = (
        number_of_chiefs * o_chief + number_of_deputies * o_deputy
    )  # Global observation vector dimension

    # Observation noise
    r_chief_pos: float = 1e-1  # [m]
    R_chief: np.ndarray = np.diag(np.concatenate([r_chief_pos * np.ones(o_chief)])) ** 2
    r_deputy_pos: float = 1e0  # [m]
    R_deputy: np.ndarray = (
        np.diag(np.concatenate([r_deputy_pos * np.ones(o_deputy)])) ** 2
    )
    R_deputies: np.ndarray = block_diag(R_deputy, R_deputy, R_deputy, R_deputy, R_deputy, R_deputy, R_deputy, R_deputy, R_deputy)
    R: np.ndarray = block_diag(R_chief, R_deputies)

    # Initial deviation noise
    # Warm-start parameters
    p_pos_initial: float = 1e3  # [m]
    p_vel_initial: float = 1e1  # [m / s]
    # Cold-start parameters
    # p_pos_initial: float = 1e2  # [m]
    # p_vel_initial: float = 1e0  # [m / s]
    P_0_spacecraft: np.ndarray = (
        np.diag(
            np.concatenate([p_pos_initial * np.ones(n_p), p_vel_initial * np.ones(n_v)])
        )
        ** 2
    )
    P_0: np.ndarray = block_diag(
        P_0_spacecraft, P_0_spacecraft, P_0_spacecraft, P_0_spacecraft,
        P_0_spacecraft, P_0_spacecraft, P_0_spacecraft, P_0_spacecraft,
        P_0_spacecraft, P_0_spacecraft
    )

    # Levenberg-Marquardt parameters
    lambda_0: float = 1.0
    epsilon: float = 1e-6
    max_iter: int = 100

    # Consensus parameters
    L: int = 1  # Number of consensus iterations
    gamma: float = N  # Consensus gain 1
    pi: float = 1 / N  # Consensus gain 2

    # Newton's method-based algorithms parameters
    grad_norm_order_mag: bool = True
    grad_norm_tol: float = 1e-6
    max_iterations: int = 20

    # Majorization-Minimization parameters
    mm_tol: float = 1e0
    mm_max_iter: int = 20

    # Post-processing parameters
    invalid_rmse: float = 1e2  # [m]
    K_RMSE: int = H  # Index from which the RMSE is calculated

    # Spacecraft parameters
    mass: float = 1.0  # Mass [kg]
    C_drag: float = 2.2  # Drag coefficient
    A_drag: float = 0.01  # Drag area [m^2]
    C_SRP: float = 1.2  # SRP coefficient
    A_SRP: float = 0.01  # SRP area [m^2]

    # Observation model
    @classmethod
    def h(cls, x_vec):
        p_vecs = [x_vec[i : i + cls.n_p] for i in range(0, cls.n, cls.n_x)]
        distances = [p_vecs[i] - p_vecs[j] for (i, j) in [(1, 0), (2, 0), (3, 1), (4, 1), (5, 2), (6, 2), (7, 3), (8, 3), (9, 5)]]
        return np.concatenate((p_vecs[0], np.array(distances).reshape(-1, 1)))

    @classmethod
    def Dh(cls, x_vec):
        first_order_der = np.zeros((cls.o, cls.n))

        first_order_der[: cls.n_p, : cls.n_p] = np.eye(cls.n_p)

        for k, (i, j) in enumerate([(1, 0), (2, 0), (3, 1), (4, 1), (5, 2), (6, 2), (7, 3), (8, 3), (9, 5)], start=1):
            first_order_der[
                cls.n_p * k : cls.n_p * (k + 1), i * cls.n_x : i * cls.n_x + cls.n_p
            ] = np.eye(cls.n_p)
            first_order_der[
                cls.n_p * k : cls.n_p * (k + 1), j * cls.n_x : j * cls.n_x + cls.n_p
            ] = -np.eye(cls.n_p)

        return first_order_der

    @classmethod
    def Hh(cls, x_vec):
        return np.zeros((cls.o * cls.n, cls.n))

In [47]:
config = SimulationConfig()

In [48]:
class Newton:
    def __init__(self):
        # Simulation parameters
        self.H = config.H
        self.K = config.K
        self.dt = config.dt
        self.o = config.o
        self.R = config.R
        self.dyn = Dynamics()

        # Stopping criteria
        self.grad_norm_order_mag = config.grad_norm_order_mag
        self.grad_norm_tol = config.grad_norm_tol
        self.max_iterations = config.max_iterations

        # Storage for results
        self.iterations = None
        self.cost_values = []
        self.gradient_norm_values = []
        self.grad_norm_order_history = []
        self.HJ_x_eigenvalues_history = []
        self.A_norm_history, self.B_norm_history, self.C_norm_history = [], [], []

        # Observation model
        self.h = config.h
        self.Dh = config.Dh
        self.Hh = config.Hh

    def J(self, k, Y, x_vec):
        if k < self.H - 1 or k + 1 > self.K:
            raise ValueError("k is out of bounds")
        R_inv = np.linalg.inv(self.R)
        J_x = 0
        for tau in range(k - self.H + 1, k + 1):
            y = Y[:, :, tau]
            h_x = self.h(x_vec)
            J_x += 1 / 2 * (y - h_x).T @ R_inv @ (y - h_x)
            x_vec = self.dyn.f(self.dt, x_vec)
        return J_x

    def DJ(self, k, Y, x_vec):
        if k < self.H - 1 or k + 1 > self.K:
            raise ValueError("k is out of bounds")
        R_inv = np.linalg.inv(self.R)
        STM = np.eye(config.n)
        DJ_x = np.zeros((config.n, 1))
        for tau in range(k - self.H + 1, k + 1):
            y = Y[:, :, tau]
            DJ_x += -STM.T @ self.Dh(x_vec).T @ R_inv @ (y - self.h(x_vec))
            STM = self.dyn.Df(self.dt, x_vec) @ STM
            x_vec = self.dyn.f(self.dt, x_vec)
        return DJ_x

    def HJ(self, k, Y, x_vec):
        if k < self.H - 1 or k + 1 > self.K:
            raise ValueError("k is out of bounds")
        R_inv = np.linalg.inv(self.R)
        STM = np.eye(config.n)
        DSTM = np.zeros((config.n * config.n, config.n))
        HJ_x = np.zeros((config.n, config.n))
        for tau in range(k - self.H + 1, k + 1):
            y = Y[:, :, tau]
            h_x = self.h(x_vec)
            Dh_x = self.Dh(x_vec)
            Hh_x = self.Hh(x_vec)
            Df_x = STM
            Hf_x = DSTM
            HJ_x += (
                -(
                    np.kron(R_inv @ (y - h_x), Df_x).T @ Hh_x @ Df_x
                    + np.kron(Dh_x.T @ R_inv @ (y - h_x), np.eye(config.n)).T @ Hf_x
                )
                + Df_x.T @ Dh_x.T @ R_inv @ Dh_x @ Df_x
            )
            DSTM = (
                np.kron(np.eye(config.n), STM).T @ self.dyn.Hf(self.dt, x_vec) @ STM
                + np.kron(self.dyn.Df(self.dt, x_vec), np.eye(config.n)) @ DSTM
            )
            STM = self.dyn.Df(self.dt, x_vec) @ STM
            x_vec = self.dyn.f(self.dt, x_vec)
        return HJ_x

    def solve_MHE_problem(self, k, Y, x_init, x_true_initial, x_true_end):
        x = x_init.copy()

        prev_cost_value = None
        prev_gradient_norm_value = None
        prev_global_estimation_error = None
        grad_norm_order_history = []

        for iteration in range(self.max_iterations + 1):
            # Compute the cost function, gradient of the Lagrangian and Hessian of the Lagrangian
            J_x = self.J(k, Y, x)
            DJ_x = self.DJ(k, Y, x)
            HJ_x = self.HJ(k, Y, x)

            # Convergence tracking
            cost_value = J_x[0][0]
            gradient_norm_value = np.linalg.norm(DJ_x)

            # Store the values
            self.cost_values.append(cost_value)
            self.gradient_norm_values.append(gradient_norm_value)

            # Metrics
            if prev_cost_value is not None:
                cost_value_change = (
                    (cost_value - prev_cost_value) / abs(prev_cost_value) * 100
                )
                gradient_norm_value_change = (
                    (gradient_norm_value - prev_gradient_norm_value)
                    / abs(prev_gradient_norm_value)
                    * 100
                )
                global_estimation_error_change = (
                    (np.linalg.norm(x - x_true_initial) - prev_global_estimation_error)
                    / abs(prev_global_estimation_error)
                    * 100
                )
            prev_cost_value = cost_value
            prev_gradient_norm_value = gradient_norm_value
            prev_global_estimation_error = np.linalg.norm(x - x_true_initial)

            # Track gradient norm order of magnitude
            current_order = int(
                np.floor(np.log10(gradient_norm_value + 1e-12))
            )  # avoid log(0)
            grad_norm_order_history.append(current_order)

            if self.grad_norm_order_mag:
                if len(grad_norm_order_history) >= 3:
                    if (
                        grad_norm_order_history[-1]
                        == grad_norm_order_history[-2]
                        == grad_norm_order_history[-3]
                    ):
                        stagnant_order = True
                        if k == self.H - 1:
                            stagnant_order = False
                    else:
                        stagnant_order = False
                else:
                    stagnant_order = False
            else:
                stagnant_order = False

            # Propagate window initial conditions for metrics
            x_end = x.copy()
            for _ in range(self.H - 1):
                x_end = self.dyn.f(self.dt, x_end)

            # Check convergence and print metrics
            if (
                gradient_norm_value < self.grad_norm_tol
                or iteration == self.max_iterations
                or stagnant_order
            ):
                reason = (
                    "tolerance reached"
                    if gradient_norm_value < self.grad_norm_tol
                    else (
                        "max iteration reached"
                        if iteration == self.max_iterations
                        else "gradient norm stagnated"
                    )
                )
                print(f"[Newton] STOP on Iteration {iteration} ({reason})")
                print(
                    f"Cost function = {cost_value} ({cost_value_change:.2f}%)\nGradient norm = {gradient_norm_value} ({gradient_norm_value_change:.2f}%)\nGlobal estimation error = {np.linalg.norm(x - x_true_initial)} ({global_estimation_error_change:.2f}%)"
                )
                print(
                    f"Final initial conditions estimation errors: {np.linalg.norm(x[:config.n_p, :] - x_true_initial[:config.n_p, :])} m, {np.linalg.norm(x[config.n_x : config.n_x + config.n_p, :] - x_true_initial[config.n_x : config.n_x + config.n_p, :])} m, {np.linalg.norm(x[2 * config.n_x : 2 * config.n_x + config.n_p, :] - x_true_initial[2 * config.n_x : 2 * config.n_x + config.n_p, :])} m, {np.linalg.norm(x[3 * config.n_x : 3 * config.n_x + config.n_p, :] - x_true_initial[3 * config.n_x : 3 * config.n_x + config.n_p, :])} m"
                )
                print(
                    f"Final position estimation errors: {np.linalg.norm(x_end[:config.n_p, :] - x_true_end[:config.n_p, :])} m, {np.linalg.norm(x_end[config.n_x : config.n_x + config.n_p, :] - x_true_end[config.n_x : config.n_x + config.n_p, :])} m, {np.linalg.norm(x_end[2 * config.n_x : 2 * config.n_x + config.n_p, :] - x_true_end[2 * config.n_x : 2 * config.n_x + config.n_p, :])} m, {np.linalg.norm(x_end[3 * config.n_x : 3 * config.n_x + config.n_p, :] - x_true_end[3 * config.n_x : 3 * config.n_x + config.n_p, :])} m\n"
                )
                break
            else:
                if iteration == 0:
                    print(
                        f"[Newton] Before applying the algorithm\nCost function: {cost_value}\nGradient norm: {gradient_norm_value}\nGlobal estimation error: {np.linalg.norm(x - x_true_initial)}"
                    )
                else:
                    print(
                        f"[Newton] Iteration {iteration}\nCost function: {cost_value} ({cost_value_change:.2f}%)\nGradient norm: {gradient_norm_value} ({gradient_norm_value_change:.2f}%)\nGlobal estimation error: {np.linalg.norm(x - x_true_initial)} ({global_estimation_error_change:.2f}%)"
                    )

            # Print estimation errors
            print(
                f"Initial conditions estimation errors: {np.linalg.norm(x[:config.n_p, :] - x_true_initial[:config.n_p, :])} m, {np.linalg.norm(x[config.n_x : config.n_x + config.n_p, :] - x_true_initial[config.n_x : config.n_x + config.n_p, :])} m, {np.linalg.norm(x[2 * config.n_x : 2 * config.n_x + config.n_p, :] - x_true_initial[2 * config.n_x : 2 * config.n_x + config.n_p, :])} m, {np.linalg.norm(x[3 * config.n_x : 3 * config.n_x + config.n_p, :] - x_true_initial[3 * config.n_x : 3 * config.n_x + config.n_p, :])} m"
            )
            print(
                f"Position estimation errors: {np.linalg.norm(x_end[:config.n_p, :] - x_true_end[:config.n_p, :])} m, {np.linalg.norm(x_end[config.n_x : config.n_x + config.n_p, :] - x_true_end[config.n_x : config.n_x + config.n_p, :])} m, {np.linalg.norm(x_end[2 * config.n_x : 2 * config.n_x + config.n_p, :] - x_true_end[2 * config.n_x : 2 * config.n_x + config.n_p, :])} m, {np.linalg.norm(x_end[3 * config.n_x : 3 * config.n_x + config.n_p, :] - x_true_end[3 * config.n_x : 3 * config.n_x + config.n_p, :])} m\n"
            )

            # Solve for the Newton step - this is one iteration
            delta_x = solve(HJ_x, -DJ_x)
            x += delta_x

            # Save the current iteration
            self.iterations = iteration + 1

        # Propagate window initial conditions getting estimate at timestamp k
        x_init = x
        for _ in range(self.H - 1):
            x = self.dyn.f(self.dt, x)

        return x_init, x

In [49]:
def get_form_initial_conditions():
    # Nominal (chief) orbit parameters
    semi_major_axis = 6903.50e3  # meters
    eccentricity = 0.0011
    inclination = np.deg2rad(97.49)
    arg_periapsis = 0.0
    raan = 0.0
    true_anomaly = 0.0

    # Chief satellite (inertial Cartesian state)
    x_initial_chief = coe2rv(
        semi_major_axis=semi_major_axis,
        eccentricity=eccentricity,
        inclination=inclination,
        argument_of_periapsis=np.deg2rad(arg_periapsis),
        longitude_of_ascending_node=np.deg2rad(raan),
        true_anomaly=np.deg2rad(true_anomaly),
    ).reshape(6, 1)

    # List of small displacements in RIC/RTN frame [R, T, N] in meters
    relative_positions_rtn = [
        [50, 0, 0],
        [-50, 0, 0],
        [0, 50, 0],
        [0, -50, 0],
        [0, 0, 50],
        [0, 0, -50],
        [25, 25, 0],
        [-25, -25, 0],
        [10, -10, 10],
    ]

    deputies = []
    for dr, dt, dn in relative_positions_rtn:
        # Convert RTN displacement to ECI frame
        r_chief = x_initial_chief[:3].flatten()
        v_chief = x_initial_chief[3:].flatten()
        r_hat = r_chief / np.linalg.norm(r_chief)
        h_vec = np.cross(r_chief, v_chief)
        h_hat = h_vec / np.linalg.norm(h_vec)
        t_hat = np.cross(h_hat, r_hat)
        R_rtn_to_eci = np.column_stack((r_hat, t_hat, h_hat))

        delta_r_eci = R_rtn_to_eci @ np.array([dr, dt, dn])
        delta_v_eci = np.zeros(3)  # no velocity offset

        deputy_state = np.hstack((r_chief + delta_r_eci, v_chief + delta_v_eci)).reshape(6, 1)
        deputies.append(deputy_state)

    all_states = [x_initial_chief] + deputies
    return np.vstack(all_states)

In [50]:
formation = 3

# Initial conditions and true state vectors (from Tudat)
X_initial = get_form_initial_conditions()
with open(f"../data/tudatpy_form{formation}_ts_{int(config.dt)}.pkl", "rb") as file:
    X_true = pickle.load(file)
X_true = X_true[:, :, : config.K]  # Truncate the true state vector to the simulation duration
    
dynamics_propagator = Dynamics()

# Set the random seed for reproducibility
if config.seed is not None:
    np.random.seed(config.seed)

In [51]:
newton = Newton()

# Generate observations
Y = np.zeros((config.o, 1, config.K))
for k in range(config.K):
    Y[:, :, k] = config.h(X_true[:, :, k]) + np.random.multivariate_normal(np.zeros(config.o), config.R).reshape((config.o, 1))

# Initial guess for the state vector
X_est = np.full_like(X_true, np.nan)
x_init = X_initial + np.random.multivariate_normal(np.zeros(config.n), config.P_0).reshape((config.n, 1))

# Run the framework
for k in tqdm(range(config.H - 1, config.K), desc="Windows", leave=False):
    x_init, x_est_k = newton.solve_MHE_problem(k, Y, x_init, X_true[:, :, k - config.H + 1], X_true[:, :, k])
    X_est[:, :, k] = x_est_k
    x_init = dynamics_propagator.f(config.dt, x_init)  # Warm-start for the next MHE problem

Windows:   0%|          | 0/2369 [00:00<?, ?it/s]

[Newton] Before applying the algorithm
Cost function: 350395846.072637
Gradient norm: 1853840.3598249613
Global estimation error: 5396.547760824044
Initial conditions estimation errors: 1713.8119175509528 m, 848.4071788093362 m, 1009.0437593751351 m, 2243.528284839653 m
Position estimation errors: 1772.3528851339026 m, 692.6359870047658 m, 1131.1820705737052 m, 2186.1706138955815 m

[Newton] Iteration 1
Cost function: 6.516059901687807e-07 (-100.00%)
Gradient norm: 0.06273886302398939 (-100.00%)
Global estimation error: 7.189788401063843 (-99.87%)
Initial conditions estimation errors: 0.07658446022505336 m, 0.8407366248297042 m, 1.6099814662826661 m, 2.193085019322614 m
Position estimation errors: 0.10760852739512831 m, 1.9905018347082153 m, 1.7829427324285843 m, 1.3165809836085223 m



Windows:   0%|          | 1/2369 [00:07<5:13:15,  7.94s/it]

[Newton] STOP on Iteration 2 (tolerance reached)
Cost function = 7.926375222265969e-18 (-100.00%)
Gradient norm = 7.31407989563745e-08 (-100.00%)
Global estimation error = 7.190255529137863 (0.01%)
Final initial conditions estimation errors: 0.07661599595643208 m, 0.8410465881390773 m, 1.6097718178284812 m, 2.1938471283266385 m
Final position estimation errors: 0.1075656101737812 m, 1.9904858339852658 m, 1.782995170686607 m, 1.316414788839045 m

[Newton] Before applying the algorithm
Cost function: 70.23620453861287
Gradient norm: 322.4376279270882
Global estimation error: 6.992950191982769
Initial conditions estimation errors: 0.1075656101737812 m, 1.9904858339852658 m, 1.782995170686607 m, 1.316414788839045 m
Position estimation errors: 0.2956667751978278 m, 4.3936749689813785 m, 4.712907717169247 m, 4.05914773166645 m



Windows:   0%|          | 2/2369 [00:13<4:24:27,  6.70s/it]

...

Windows:   0%|          | 3/2369 [00:19<4:01:10,  6.12s/it]

[Newton] STOP on Iteration 1 (tolerance reached)
Cost function = 1.8552777589884935e-18 (-100.00%)
Gradient norm = 3.3609034094867225e-08 (-100.00%)
Global estimation error = 5.961700676542658 (0.15%)
Final initial conditions estimation errors: 0.08075683412022232 m, 1.170116207493238 m, 1.97775133075642 m, 1.155196213098751 m
Final position estimation errors: 0.17992375138886685 m, 1.2514297134801533 m, 1.483991604127284 m, 1.5063916704160167 m

[Newton] Before applying the algorithm
Cost function: 92.87520458337812
Gradient norm: 374.70201008955377
Global estimation error: 5.837225143894978
Initial conditions estimation errors: 0.17992375138886685 m, 1.2514297134801533 m, 1.483991604127284 m, 1.5063916704160167 m
Position estimation errors: 0.4280049541177618 m, 2.4347741208175724 m, 4.390950155751664 m, 2.243733732885621 m



Windows:   0%|          | 4/2369 [00:24<3:48:22,  5.79s/it]

[Newton] STOP on Iteration 1 (tolerance reached)
Cost function = 2.680098345521458e-18 (-100.00%)
Gradient norm = 3.654128829099898e-08 (-100.00%)
Global estimation error = 5.904302781910511 (1.15%)
Final initial conditions estimation errors: 0.17992375138655614 m, 1.2514297134329313 m, 1.483991604377007 m, 1.5063916703663596 m
Final position estimation errors: 0.1450595931229163 m, 1.7947364251543625 m, 2.773140792720511 m, 2.117479116784588 m



KeyboardInterrupt: 

In [ ]:
from utils import save_simulation_data

class Args:
    monte_carlo_sims = 1
    algorithm = "newton"
    formation = formation
    
args = Args()
X_est_all = []
X_est_all.append(X_est)
save_simulation_data(args, X_true, X_est_all)